# Keras MLP 
## Overview ##
This kernel uses seeding and relative seeding with an MLP network. Spread on loss was very small, so kfold not necessary

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from time import localtime
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras import regularizers
from keras.callbacks import EarlyStopping, TensorBoard, ProgbarLogger
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV

from sklearn import ensemble
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../NCAA"]).decode("utf8"))

# Any results you write to the current directory are saved as output.


### Lock Random Seed

In [ ]:
seed = 13
np.random.seed(seed)

# Load and Organize Training data


In [ ]:
data_dir = '../NCAA/Data/'
df_seeds = pd.read_csv(data_dir + 'NCAATourneySeeds.csv')
df_tour = pd.read_csv(data_dir + 'NCAATourneyCompactResults.csv')
df_tour.head()

Incorporate Massey Ordinals - MAS, SAG, POM

In [ ]:
#Load Data
df_rank = pd.read_csv(data_dir+ 'MasseyOrdinals.csv')
#Choose Latest Ranking 
df_rank = df_rank[df_rank.RankingDayNum>=133]
df_rank = df_rank[df_rank.Season>=2010]

#Selectr rankings of interest and make each ranking system ranking into a separate column
df_merge = pd.merge(df_rank.loc[df_rank['SystemName']=='MAS'],
                    df_rank.loc[df_rank['SystemName']=='SAG'], how='left', 
                    on=['Season', 'TeamID', 'RankingDayNum'])
df_rank = pd.merge(left=df_merge, right=df_rank.loc[df_rank['SystemName']=='POM'], 
                  how='left', on=['Season', 'TeamID', 'RankingDayNum'] )

Remove unnecessary cols

In [ ]:
# Drop
df_rank.drop(labels=['SystemName_x', 'SystemName_y', 'SystemName','RankingDayNum'], inplace=True, axis =1)
df_rank.rename(columns={'OrdinalRank_x':'MAS', 'OrdinalRank_y':'SAG', 'OrdinalRank':'POM'}, inplace=True)

# df_rank.head()

Add advanced stats from https://www.kaggle.com/lnatml/feature-engineering-with-advanced-stats/notebook

In [ ]:
df = pd.read_csv('../input/NCAATourneyDetailedResults.csv')

#Points Winning/Losing Team
df_adv['WPts'] = df.apply(lambda row: 2*(row.WFGM-row.WFGM3) + 3*row.WFGM3 + row.WFTM, axis=1)
df_adv['LPts'] = df.apply(lambda row: 2*(row.LFGM-row.WFGM3) + 3*row.LFGM3 + row.LFTM, axis=1)

#Calculate Winning/losing Team Possesion Feature
wPos = df.apply(lambda row: 0.96*(row.WFGA + row.WTO + 0.44*row.WFTA - row.WOR), axis=1)
lPos = df.apply(lambda row: 0.96*(row.LFGA + row.LTO + 0.44*row.LFTA - row.LOR), axis=1)
#two teams use almost the same number of possessions in a game
#(plus/minus one or two - depending on how quarters end)
#so let's just take the average
df_adv['Pos'] = (wPos+lPos)/2

In [ ]:
#Offensive efficiency (OffRtg) = 100 x (Points / Possessions)
df_adv['WOffRtg'] = df.apply(lambda row: 100 * (row.WPts / row.Pos), axis=1)
df_adv['LOffRtg'] = df.apply(lambda row: 100 * (row.LPts / row.Pos), axis=1)
#Defensive efficiency (DefRtg) = 100 x (Opponent points / Opponent possessions)
df_adv['WDefRtg'] = df.LOffRtg
df_adv['LDefRtg'] = df.WOffRtg
#Net Rating = Off.eff - Def.eff
df_adv['WNetRtg'] = df.apply(lambda row:(row.WOffRtg - row.LDefRtg), axis=1)
df_adv['LNetRtg'] = df.apply(lambda row:(row.LOffRtg - row.LDefRtg), axis=1)
                         
#Assist Ratio : Percentage of team possessions that end in assists
df_adv['WAstR'] = df.apply(lambda row: 100 * row.WAst / (row.WFGA + 0.44*row.WFTA + row.WAst + row.WTO), axis=1)
df_adv['LAstR'] = df.apply(lambda row: 100 * row.LAst / (row.LFGA + 0.44*row.LFTA + row.LAst + row.LTO), axis=1)
#Turnover Ratio: Number of turnovers of a team per 100 possessions used.
#(TO * 100) / (FGA + (FTA * 0.44) + AST + TO
df_adv['WTOR'] = df.apply(lambda row: 100 * row.LAst / (row.LFGA + 0.44*row.LFTA + row.LAst + row.LTO), axis=1)
df_adv['LTOR'] = df.apply(lambda row: 100 * row.LAst / (row.LFGA + 0.44*row.LFTA + row.LAst + row.LTO), axis=1)
                    
#The Shooting Percentage : Measure of Shooting Efficiency (FGA/FGA3, FTA)
df_adv['WTSP'] = df.apply(lambda row: 100 * row.WPts / (2 * (row.WFGA + 0.44 * row.WFTA)), axis=1)
df_adv['LTSP'] = df.apply(lambda row: 100 * row.LPts / (2 * (row.LFGA + 0.44 * row.LFTA)), axis=1)
#eFG% : Effective Field Goal Percentage adjusting for the fact that 3pt shots are more valuable 
df_adv['WeFGP'] = df.apply(lambda row:(row.WFGM + 0.5 * row.WFGM3) / row.WFGA, axis=1)      
df_adv['LeFGP'] = df.apply(lambda row:(row.LFGM + 0.5 * row.LFGM3) / row.LFGA, axis=1)   
#FTA Rate : How good a team is at drawing fouls.
df_adv['WFTAR'] = df.apply(lambda row: row.WFTA / row.WFGA, axis=1)
df_adv['LFTAR'] = df.apply(lambda row: row.LFTA / row.LFGA, axis=1)
                         
#OREB% : Percentage of team offensive rebounds
df_adv['WORP'] = df.apply(lambda row: row.WOR / (row.WOR + row.LDR), axis=1)
df_adv['LORP'] = df.apply(lambda row: row.LOR / (row.LOR + row.WDR), axis=1)
#DREB% : Percentage of team defensive rebounds
df_adv['WDRP'] = df.apply(lambda row: row.WDR / (row.WDR + row.LOR), axis=1)
df_adv['LDRP'] = df.apply(lambda row: row.LDR / (row.LDR + row.WOR), axis=1)                                      
#REB% : Percentage of team total rebounds
df_adv['WRP'] = df.apply(lambda row: (row.WDR + row.WOR) / (row.WDR + row.WOR + row.LDR + row.LOR), axis=1)
df_adv['LRP'] = df.apply(lambda row: (row.LDR + row.WOR) / (row.WDR + row.WOR + row.LDR + row.LOR), axis=1) 

Cut seasons prior to Max Ranking date

In [ ]:
df_seeds = df_seeds[df_seeds.Season>=min(df_rank.Season) ]
df_seeds = df_seeds[ df_seeds.Season<=max(df_rank.Season)]
df_tour = df_tour[df_tour.Season>=min(df_rank.Season)]
df_tour = df_tour[ df_tour.Season<=max(df_rank.Season)]
# df_tour.head()

Cut off the region identifier from the seed number

In [ ]:
def seed_to_int(seed):
    #Get just the digits from the seeding. Return as int
    s_int = int(seed[1:3])
    return s_int
df_seeds['seed_int'] = df_seeds.Seed.apply(seed_to_int)
df_seeds.drop(columns=['Seed'], inplace=True) # This is the string label

df_tour.drop(labels=['DayNum', 'WScore', 'LScore','WLoc', 'NumOT'], inplace=True, axis=1)
# df_tour.head()

Merge the Seeds with their corresponding TeamIDs in the compact results dataframe.

In [ ]:
df_winSeeds = df_seeds.rename(columns={'TeamID':'WTeamID', 'seed_int':'WSeed'})
df_lossSeeds = df_seeds.rename(columns={'TeamID':'LTeamID', 'seed_int':'LSeed'})


df_dummy = pd.merge(left=df_tour, right=df_lossSeeds, how='left', on=['Season', 'LTeamID'])

df_concat = pd.merge(left=df_dummy, right=df_winSeeds, how='left' ,on=['Season', 'WTeamID'])
# df_concat['SeedDiff'] = df_concat.WSeed - df_concat.LSeed
# df_concat.head()

 Make a winner and loser DF with seed, relative seed, and win/loss. 

In [ ]:
df_wins = pd.DataFrame()
df_wins['Seed'] = df_concat['WSeed']
# df_wins['SeedDiff'] = df_concat['SeedDiff']
df_wins['TeamID'] = df_concat['WTeamID']
df_wins['Season'] = df_concat['Season']
df_wins['Result'] = 1
df_wins = pd.merge(left=df_wins, right=df_rank, how='left', on=['Season', 'TeamID'])
# df_wins.head()

In [ ]:
df_losses = pd.DataFrame()
df_losses['Seed'] = df_concat['LSeed']
# df_losses['SeedDiff'] = -df_concat['SeedDiff']
df_losses['TeamID'] = df_concat['LTeamID']
df_losses['Season'] = df_concat['Season']
df_losses['Result'] = 0
df_losses = pd.merge(left=df_losses, right=df_rank, how='left',on=['Season', 'TeamID'])
# df_losses.head()

Prefix the cols with 'Opp' in dummy frames to allow for horizontal concat in order to expand data to be

| winr_stat    losr_stat |
                     
| losr_stat    winr_stat |

In [ ]:
df_lossesOpp = df_losses.copy()
df_lossesOpp.drop(labels=['Season', 'Result'], inplace=True, axis=1)
new_names = [(i,'Opp'+i) for i in df_lossesOpp.columns.values]
df_lossesOpp.rename(columns = dict(new_names), inplace=True)
# df_lossesOpp.head()

df_winsOpp = df_wins.copy()
df_winsOpp.drop(labels=['Season', 'Result'], inplace=True, axis=1)
new_names = [(i,'Opp'+i) for i in df_winsOpp.columns.values]
df_winsOpp.rename(columns = dict(new_names), inplace=True)
# df_winsOpp.head()

df_winloss = pd.concat([df_wins, df_lossesOpp], axis=1)

df_losswin = pd.concat([df_losses, df_winsOpp], axis=1)
# df_losswin.head()

Combine into final dataframe

In [ ]:
df_finalData = pd.concat((df_winloss, df_losswin))
results = df_finalData['Result']
df_finalData.drop(labels=['Result'], inplace=True, axis=1)
df_finalData.insert(0, 'Result', results)
# df_finalData.head()

Remove team ID and season -> cannot be scaled and not a metric. Dummy

In [ ]:
df_finalData.drop(labels=['TeamID', 'Season', 'OppTeamID'], inplace=True, axis=1)
df_finalData.head()

df_final.to_csv('MarchMadnessFeatures', index=False)

In [ ]:
df_finalData.isnull().any()

# Fit the Model 

###  StandardScaler


In [ ]:
X = df_finalData.iloc[:,1:]
xDim = np.shape(X)[1]
X= X.values.reshape(-1,xDim)
y = df_finalData.Result.values

scaler = StandardScaler()
X = scaler.fit_transform(X)

print('Feature vector dimension is: %.2f' % xDim)
print(X[:5,:])


###  Random Forest Classifier

In [ ]:
# Fit the model

fore = RandomForestClassifier(criterion='entropy', n_estimators = 20, max_depth = 4, random_state=0)

fore.fit(X, y)

In [ ]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
cvresults = cross_val_score(fore, X, y, cv=kfold ,scoring='neg_log_loss', verbose=0)
print('Log-Loss Mean :{:.3} ({:.3})'.format(np.mean(cvresults), np.std(cvresults)))

In [ ]:
# boxplot algorithm comparison
# fig = plt.figure()
# fig.suptitle('Algorithm Comparison')
# plt.boxplot((cvresults))
# plt.show()


### Extract the data desired for the submission

In [ ]:
df_sample_sub = pd.read_csv(data_dir + 'SampleSubmissionStage1.csv')
n_test_games = len(df_sample_sub)

def get_year_t1_t2(ID):
    """Return a tuple with ints `year`, `team1` and `team2`."""
    return (int(x) for x in ID.split('_'))

In [ ]:
X_test = np.zeros(shape=(n_test_games, xDim))
for ii, row in df_sample_sub.iterrows():
    year, t1, t2 = get_year_t1_t2(row.ID)
    t1_seed = df_seeds[(df_seeds.TeamID == t1) & (df_seeds.Season == year)].seed_int.values[0]
    t2_seed = df_seeds[(df_seeds.TeamID == t2) & (df_seeds.Season == year)].seed_int.values[0]
    diff_seed = t1_seed - t2_seed
    X_test[ii, 0] = t1_seed
    X_test[ii, 1] = diff_seed
    X_test[ii, 2] = t1
    X_test[ii, 3] = year


### Scale data with appropriate scaler

In [ ]:
X_test = scaler.fit_transform(X_test)

## Make Predictions ##
Create predictions using the logistic regression model we trained.

In [ ]:
preds = fore.predict_proba(X_test)

clipped_preds = np.clip(preds, 0.05, 0.95)
df_sample_sub.Pred = clipped_preds
df_sample_sub.head()

Lastly, create your submission file!

In [ ]:
filename = 'RandomForest_RankSeeds'
c=0
ext = '.csv'
if os.path.exists(filename+ext):
    while os.path.exists(filename+ext):
        c+=1
        filename = filename+'_'+str(c)
    df_sample_sub.to_csv(filename+ext, index=False)
else:
    df_sample_sub.to_csv(filename+ext, index=False)